In [5]:
import os
import requests
from ollama import chat
from ollama import ChatResponse
from firecrawl import FirecrawlApp
from pydantic import BaseModel
from typing import Optional
from dotenv import load_dotenv
from IPython.display import display, Markdown
from requests.exceptions import HTTPError
load_dotenv()

SERP_API_KEY = os.getenv("SERP_API_KEY")
FIRECRAWL_API_KEY = os.getenv("FIRECRAWL_API_KEY")
MODEL = os.getenv("MODEL")

In [6]:
def fetch_serp(q: str):
   url = 'https://serpapi.com/search'
   params = {
        'api_key': SERP_API_KEY,
        'engine': 'google',
        'q': q,
        'google_domain': 'google.com',
        'hl': 'en'
    }
   response = requests.get(url, params=params)
   if response.status_code == 200:
        json_response = response.json()
        organic = json_response['organic_results']
        formatted_results = []
        for result in organic:
            title = result['title']
            link = result['link']
            description = result['snippet']
            resource_object = {
                'title': title,
                'link': link,
                'description': description
            }
            formatted_results.append(resource_object)
        return formatted_results

In [7]:
app = FirecrawlApp(api_key=FIRECRAWL_API_KEY)
def retrieve_context(url:str):
    context = app.scrape_url(url, params={'formats':['markdown']})
    return context

In [8]:
class SearchQueries(BaseModel):
    queries: list[str] 

user_prompt = input("Topic of Research: ")
research_objective = input("Research objective: ")
research_width = input("Width of Research (1-5): ")

def generate_queries(prompt:str, objective:str, width:str):
    generated_queries: ChatResponse = chat(model=MODEL, messages=[
    {
        'role': 'user',
        'content': f"Given the user's prompt, generate {width} search queries (to be used as Google Search queries) that will get you started with performing deep research on the topic in hand. It is important that the queries you generate are unique and have ZERO overlap with each other. The number of search queries you are asked to generate defines the width of the user's research. Do not generate anything else apart from the {width} queries required. Here's the user's research title:{prompt}. And here's the research objective: {objective}. Your thought process while generating these queries should consider the main objective of the research, and you should figure out how to progressively.",
    },
    ],
    format=SearchQueries.model_json_schema())
    formatted_queries = SearchQueries.model_validate_json(generated_queries.message.content)
    return formatted_queries

In [9]:
search_results = []
current_queries = generate_queries(user_prompt, research_objective, research_width)
for q in current_queries.queries:
    # Iteratively fetch Google Search results for each generated search query
    results = fetch_serp(q)
    search_results.extend(results)

In [10]:
class ResultItem(BaseModel):
    title: str
    link: str

class RelevantQueries(BaseModel):
    r_queries: list[ResultItem]

references = []

def evaluate_relevance(prompt:str, objective:str, width:int, serp_results):
    """This function evaluates the relevance of each search result. This is to narrow down the research to align with the user's prompt and research objective.
    Returns:
        Search results marked as relevant by the model (list[str])
        References (list[str])
    """
    relevant_queries: ChatResponse = chat(model=MODEL, messages=[
        {
            'role': 'user',
            'content': f"You are an expert data extractor. You are provided with a list of search results for a specific research query. This will be a list of objects containing the title, description and the link to an article/webpage. Use the title and description to verify if a particular search result is indeed relevant to the query. Return the titles AND links that are relevant to the research objective in the specified format. Here is the list of search results: {serp_results}. Here's the user's topic of research: {prompt}. The objective of the research is as follows: {objective}. You must return {width} results from the list that has been provided to you"
        }
    ],format=RelevantQueries.model_json_schema())
    formatted_relevant_queries = RelevantQueries.model_validate_json(relevant_queries.message.content)
    references.extend(formatted_relevant_queries.r_queries)
    return formatted_relevant_queries, references

In [11]:
research_context = []
evaluated_queries, _ = evaluate_relevance(user_prompt, research_objective, research_width, search_results)

In [12]:
for item in evaluated_queries.r_queries:
    # For each query deemed to be relevant, we hit the firecrawl API to scrape the contents of the website linked in the search result
    # TODO: Rename variables to ensure better readability
    print(f"Retrieving research context for the resource: {item.title}")
    try:
        c = retrieve_context(item.link)
        research_context.append(c.get("markdown", ""))
    except HTTPError as e:
        print(f"Failed to retrieve content from {item.title}: {str(e)}")
        references.remove(item)
        continue  # Skip to the next URL
    except Exception as e:
        print(f"Unexpected error while processing {item.title}: {str(e)}")
        continue  # Skip any other unexpected errors

Retrieving research context for the resource: Tesla Faces a Sales Storm as Consumer Shifts Drive Stock Downward
Retrieving research context for the resource: Tesla's Sales Drop 12% in California, But Remains a Dominant Force
Retrieving research context for the resource: The Global Electric Vehicle Market In 2024


In [13]:
class ContextEval(BaseModel):
    further_research: bool
    probing_queries: Optional[SearchQueries]

def evaluate_context(prompt: str, objective: str, context: list[str], width: int) -> tuple[bool, Optional[list[str]]]:
    """This function uses the language model to evaluate whether further research is needed based on the current context.
    Returns:
        Boolean (True or False)
        List of probing questions (Optional)
    """
    research_status: ChatResponse = chat(model=MODEL, messages=[
        {
            'role': 'user',
            'content': f"""
            You are an expert analyst. You are provided with information regarding the user's topic of research, which is: {prompt}.
            The objective of this research is: {objective}.
            The research context we have until now is here: {context}.
            Based on the given information, confirm if further research is required.
            However, if you think more information is required for a comprehensive report, generate {width} more probing questions
            that go deeper into the topic, but make sure the queries are specific and precise to the current context as they will be used
            to fetch Google search results, which in turn will be used for further research.
            """
        }
    ], format=ContextEval.model_json_schema())
    
    context_eval = ContextEval.model_validate_json(research_status.message.content)
    return context_eval.further_research, context_eval.probing_queries

In [14]:
def research_loop(prompt: str, objective: str, context: list[str], width: int) -> list[str]:
    """Manages the iterative research process to refine the research context. This function evaluates the existing research context initially and iterates
    a maximum of two times to generate clarifying/additional research questions if deemed necessary by the model. The max_iterations is set to 2 to avoid
    topping out your free API limits with Firecrawl and Serp. This can be altered if you plan on using your own web scraping solution.
    
    Returns:
        Updated context (list[str]) which will be used to generate the final report
    """
    max_iterations = 2  # Prevent infinite loops
    iteration_count = 0
    current_context = context.copy()
    
    while iteration_count < max_iterations:
        further_research, probing_queries = evaluate_context(prompt, objective, current_context, width)
        
        if not further_research:
            break
        
        iteration_count += 1
        new_search_results = []
        
        if probing_queries:  
            for q in probing_queries.queries:  
                new_search_results.extend(fetch_serp(q))
                
            current_evaluated_queries, refs = evaluate_relevance(prompt, objective, width, new_search_results)
            
            for _ in current_evaluated_queries.r_queries:
                print(f"Retrieving research context for the resource: {_.title}")
                try:
                    new_context = retrieve_context(_.link)
                    references.extend(_)
                    current_context.append(new_context.get("markdown", ""))
                except HTTPError as e:
                    print(f"Failed to retrieve content from {_.title}: {str(e)}")
                    references.remove(_)
                    print(f"Removed '{_}' from references")
                    continue
                except Exception as e:
                    print(f"Unexpected error while processing {_.title}: {str(e)}")
                    continue
    
    if iteration_count >= max_iterations:
        print("Max iterations reached. Proceeding with the available research context.")
    
    return current_context

In [15]:
def generate_report(prompt:str, objective:str, width: int, sources: list[str]) -> str:
    final_context = research_loop(prompt, objective, research_context, width)
    final_report: ChatResponse = chat(model=MODEL, messages=[
        {
            'role': 'user',
            'content':f"""You are an expert analyst. Your job is to use the provided research prompt, objective and context to generate a clear and comprehensive report.
            Research Prompt: {prompt}
            Research Objective: {objective}
            Research Context: {final_context}
            With this information, generate a detailed (700 words) report as instructed. Be sure to include all sources, persons, objects etc in the report. You MUST include a references section and appropriately add citations, the references are here: {sources}. The report must be in markdown format. You are allowed to take liberties with section titles and in-report formatting. Ensure that the report itself adheres to the user's requirements and does not deviate away from the research's goals. You MUST remain aligned to the research title and objective, even if the context provided contains information that may cover aspects other than the ones mentioned by the user. Use the research title provided by the user for the report's title and follow the instructions as mentioned above."""
        }
    ])
    return final_report.message.content

In [16]:
report = generate_report(user_prompt,research_objective, research_width, references)

In [17]:
display(Markdown(report))

# Report: Global Electric Vehicle Market Dynamics in 2024

## Introduction

The global electric vehicle (EV) market is undergoing significant transformations as consumer preferences shift, technological advancements occur, and regulatory frameworks evolve. This report explores key developments within this landscape in 2024, focusing on market dynamics, corporate strategies, and emerging trends that are influencing the EV sector.

## Market Trends and Consumer Preferences

In recent years, there has been a notable shift in consumer behavior towards electric vehicles, driven by environmental concerns, advancements in technology, and supportive government policies. However, despite this positive trajectory, certain challenges remain that impact sales performance across key markets.

### Tesla's Sales Performance

Tesla, a dominant player in the EV market, is experiencing fluctuations in its sales figures. According to recent reports, Tesla faced a significant sales downturn as consumer preferences began shifting away from traditional luxury brands towards more affordable and diverse options ([Tesla Faces a Sales Storm](https://www.ainvest.com/news/tesla-faces-a-sales-storm-as-consumer-shifts-drive-stock-downward-2502101085381cb03a209ffe/)). In California, Tesla’s sales dropped by 12% in recent months, although the brand continues to hold a substantial market share ([Tesla's Sales Drop](https://opentools.ai/news/teslas-sales-drop-12percent-in-california-but-remains-a-dominant-force)).

## Corporate Strategies and Innovations

Amidst competitive pressures, companies within the EV sector are actively innovating and adapting their strategies to capture market opportunities. Key areas of focus include enhancing charging infrastructure, improving battery technology, and expanding product offerings to cater to diverse consumer needs.

### Investment in Charging Infrastructure

The demand for reliable and accessible EV charging networks is on the rise. Companies like Virta Global emphasize the importance of investing in charging operations as a pivotal strategy to support market growth. By facilitating easy access to charging facilities, businesses can drive adoption rates and ensure customer satisfaction ([Virta Global](https://www.virta.global/global-electric-vehicle-market)).

### Technological Advancements

Technological innovation remains central to the EV industry's evolution. Enhanced battery technologies that offer longer ranges and faster charging times are becoming critical competitive differentiators. Additionally, advancements in vehicle connectivity and autonomous driving features are attracting tech-savvy consumers looking for cutting-edge mobility solutions.

## Regulatory Influences

Regulatory frameworks play a significant role in shaping the electric vehicle market. Governments worldwide are implementing policies to encourage EV adoption, such as subsidies, tax incentives, and mandates for zero-emission vehicles (ZEVs).

### The UK ZEV Mandate

The United Kingdom has introduced the Zero Emission Vehicle mandate, designed to accelerate the transition towards sustainable transportation. This regulatory measure is expected to have a favorable impact on businesses involved in EV manufacturing and infrastructure development by driving demand for electric models ([UK ZEV Mandate](https://www.virta.global/blog/what-is-the-uk-zev-mandate)).

## Emerging Market Opportunities

As the market matures, new opportunities are emerging across various segments. These include expanding into underserved markets, developing alternative energy solutions, and exploring partnerships with tech firms to integrate smart features into EV offerings.

### Strategic Partnerships

Strategic collaborations between automotive manufacturers and technology companies can lead to innovations in vehicle design, functionality, and user experience. Such partnerships may enhance the appeal of electric vehicles by offering integrated digital services, such as seamless connectivity and advanced driver-assistance systems.

## Conclusion

The global electric vehicle market is poised for continued growth despite current challenges. Companies that strategically invest in infrastructure, embrace technological advancements, and align with regulatory trends will likely secure a competitive advantage. As consumer preferences evolve, the industry's ability to adapt will determine its success in meeting future demands.

## References

- [Tesla Faces a Sales Storm as Consumer Shifts Drive Stock Downward](https://www.ainvest.com/news/tesla-faces-a-sales-storm-as-consumer-shifts-drive-stock-downward-2502101085381cb03a209ffe/)
- [Tesla's Sales Drop 12% in California, But Remains a Dominant Force](https://opentools.ai/news/teslas-sales-drop-12percent-in-california-but-remains-a-dominant-force)
- [The Global Electric Vehicle Market In 2024](https://www.virta.global/global-electric-vehicle-market)

This report highlights key dynamics within the global electric vehicle market in 2024, emphasizing the critical areas of consumer trends, corporate strategies, regulatory influences, and emerging opportunities. By understanding these elements, stakeholders can better navigate the evolving landscape of electric mobility.

In [18]:
with open('research_report.md', 'w', encoding='utf-8') as f:
    f.write(report)